# Semantic Kernel တွင် Agent Scratchpad ဖြင့် Chat History Reduction

ဒီ notebook က Semantic Kernel ရဲ့ Chat History Reduction လုပ်ဆောင်ချက်ကို Agent Scratchpad နဲ့ပေါင်းစပ်ပြီး စကားဝိုင်းများအတွင်း context ကို ထိန်းသိမ်းထားနိုင်ဖို့ ဘယ်လိုအသုံးပြုရမယ်ဆိုတာကို ပြသထားပါတယ်။ ဒါဟာ AI agent တွေကို အကျိုးရှိရှိ တည်ဆောက်ဖို့အတွက် အရေးကြီးပြီး၊ token အကန့်အသတ်ကို မကျော်လွန်ဘဲ ရှည်လျားတဲ့ စကားဝိုင်းတွေကို ကိုင်တွယ်နိုင်စေပါတယ်။

## သင်လေ့လာရမယ့်အရာများ:
1. **Chat History Reduction**: စကားဝိုင်းသမိုင်းကို အလိုအလျောက် အကျဉ်းချုပ်ပြီး token အသုံးပြုမှုကို စီမံခန့်ခွဲပုံ
2. **Agent Scratchpad**: အသုံးပြုသူရဲ့ အကြိုက်နှင့် ပြီးဆုံးထားသော တာဝန်များကို မှတ်သားထားနိုင်တဲ့ အမြဲတမ်းမှတ်ဉာဏ်စနစ်
3. **Token Usage Tracking**: History reduction ရှိ/မရှိဖြင့် token အသုံးပြုမှု ဘယ်လိုပြောင်းလဲသွားတယ်ဆိုတာကို စောင့်ကြည့်ခြင်း

## မလိုအပ်မဖြစ်:
- Azure OpenAI setup ကို environment variables ဖြင့် ပြင်ဆင်ထားခြင်း
- ယခင်သင်ခန်းစာများမှ agent အခြေခံအယူအဆများကို နားလည်ထားခြင်း


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## Agent Scratchpad ကိုနားလည်ခြင်း

### Agent Scratchpad ဆိုတာဘာလဲ?

**Agent Scratchpad** ဆိုတာ အေးဂျင့်တွေ အသုံးပြုတဲ့ အမြဲတမ်းမှတ်ဉာဏ်စနစ်တစ်ခုဖြစ်ပြီး:
- **ပြီးစီးထားတဲ့အလုပ်တွေကိုခြေရာခံခြင်း**: အသုံးပြုသူအတွက် ပြီးစီးထားတဲ့အလုပ်တွေကို မှတ်တမ်းတင်ထားခြင်း
- **အသုံးပြုသူအကြိုက်အနုတ်ကို သိမ်းဆည်းထားခြင်း**: အကြိုက်၊ မကြိုက်၊ လိုအပ်ချက်တွေကို မှတ်ထားခြင်း
- **အခြေအနေကို ထိန်းသိမ်းထားခြင်း**: စကားဝိုင်းတွေမှာ အရေးကြီးတဲ့အချက်အလက်တွေကို လွယ်လွယ်ကူကူရနိုင်အောင် ထိန်းသိမ်းထားခြင်း
- **ထပ်ခါတလဲလဲမေးမြန်းမှုကို လျှော့ချခြင်း**: တစ်ခါတစ်ခါ မေးရတဲ့မေးခွန်းတွေကို ထပ်မမေးဖို့ ရှောင်ရှားခြင်း

### အလုပ်လုပ်ပုံ:
1. **ရေးသားမှုလုပ်ငန်းစဉ်**: အေးဂျင့်က အသစ်ရရှိတဲ့အချက်အလက်တွေကို Scratchpad မှာ အပ်ဒိတ်လုပ်ခြင်း
2. **ဖတ်ရှုမှုလုပ်ငန်းစဉ်**: အေးဂျင့်က ဆုံးဖြတ်ချက်ချတဲ့အခါ Scratchpad ကို အကြံဉာဏ်ယူခြင်း
3. **အမြဲတမ်းရှိနေမှု**: စကားဝိုင်းသမိုင်းကို လျှော့ချရတဲ့အခါမှာတောင် အချက်အလက်တွေ မပျောက်သွားအောင် ထိန်းသိမ်းထားခြင်း

ဒါကို အေးဂျင့်ရဲ့ စကားဝိုင်းသမိုင်းကို ဖြည့်စွက်ပေးတဲ့ ကိုယ်ပိုင်မှတ်စုစာအုပ်လို့ ထင်ရနိုင်ပါတယ်။


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## အေးဂျင့် Scratchpad ပလပ်ဂင် ဖန်တီးရန်

ဒီပလပ်ဂင်က အေးဂျင့်ကို အမြဲတမ်းသိမ်းဆည်းထားတဲ့ scratchpad ဖိုင်ကို ဖတ်နိုင်ပြီး ရေးနိုင်စေပါတယ်။


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## ချတ်သမိုင်းကို လျှော့ချရန် အစပြုပါ

ChatHistorySummarizationReducer သည် စကားဝိုင်းသမိုင်းသည် သတ်မှတ်ထားသောကန့်သတ်ချက်ကို ကျော်လွန်သည့်အခါ အလိုအလျောက် အကျဉ်းချုပ်ပေးပါသည်။


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## အပန်းဖြေခရီးစဉ် စီစဉ်ရေးဆွဲသူ ဖန်တီးရန်

ဒီရေးဆွဲသူက အသုံးပြုသူများကို အပန်းဖြေခရီးစဉ်များ စီစဉ်ရာတွင် scratchpad ကို အသုံးပြု၍ အခြေအနေများကို ထိန်းသိမ်းထားနိုင်စေမှာ ဖြစ်ပါတယ်။


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## အပန်းဖြေခရီးစဉ် စီမံကိန်းဆွေးနွေးမှုကို စတင်ရန်

အခုတော့ အပြည့်အစုံသော ဆွေးနွေးမှုတစ်ခုကို ပြုလုပ်ကြမယ်၊ ဤဆွေးနွေးမှုတွင် အောက်ပါအချက်များကို ဖော်ပြပါသည်။
1. စီမံကိန်းတောင်းဆိုမှုအစ
2. အကြိုက်နှင့် အလိုဆန္ဒများ စုဆောင်းခြင်း
3. ခရီးစဉ်အစီအစဉ် ဖန်တီးခြင်း
4. တည်နေရာပြောင်းလဲခြင်း
5. စကားဝိုင်းမှတ်တမ်း လျှော့ချခြင်း
6. Scratchpad အသုံးပြုခြင်း


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## ရလဒ်များကို ခွဲခြမ်းစိတ်ဖြာခြင်း

ကျွန်တော်တို့ ဆွေးနွေးမှုအတွင်း ဘာတွေဖြစ်ခဲ့တယ်ဆိုတာ ခွဲခြမ်းစိတ်ဖြာကြည့်ရအောင်:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## အဓိကအချက်များ

### 1. စကားဝိုင်းမှတ်တမ်းလျှော့ချခြင်း
- **အလိုအလျောက်စတင်ခြင်း**: စကားဝိုင်းမက်ဆေ့ချ်အရေအတွက် သတ်မှတ်ချက်ကျော်လွန်သောအခါ လျှော့ချမှုဖြစ်ပေါ်သည်
- **Token သက်သာမှု**: အကျဉ်းချုပ်ပြီးနောက် Token အသုံးပြုမှု အလွန်လျှော့ကျသည်
- **အကြောင်းအရာထိန်းသိမ်းမှု**: အရေးကြီးသောအချက်အလက်များကို အကျဉ်းချုပ်များတွင် ထိန်းသိမ်းထားသည်

### 2. Agent Scratchpad အကျိုးကျေးဇူးများ
- **အမြဲတမ်းမှတ်ဉာဏ်**: အသုံးပြုသူ၏ အကြိုက်များသည် စကားဝိုင်းမှတ်တမ်းလျှော့ချမှုအပြီးတွင်လည်း ကျန်ရှိနေသည်
- **အလုပ်အခြေအနေမှတ်တမ်း**: Agent သည် ပြီးစီးထားသောအလုပ်များကို မှတ်တမ်းတင်ထားသည်
- **အတွေ့အကြုံတိုးတက်မှု**: အကြိုက်များကို ထပ်မံပြောရန် မလိုအပ်တော့

### 3. Token အသုံးပြုမှုပုံစံများ
- **တိုးတက်မှုတိုင်းတန်း**: မက်ဆေ့ချ်တစ်ခုစီနှင့်အတူ Token များ တိုးတက်လာသည်
- **အလွန်လျှော့ကျမှု**: လျှော့ချမှုကြောင့် Token အရေအတွက် အလွန်လျှော့ကျသည်
- **ဆက်လက်ဆွေးနွေးနိုင်မှု**: သတ်မှတ်ကန့်သတ်ချက်အတွင်း အချိန်ကြာရှည်ဆွေးနွေးမှုများကို အလွယ်တကူလုပ်ဆောင်နိုင်သည်


## ရှင်းလင်းမှု

ဒီဒေသတွင်း ဒီမိုလုပ်စဉ်အတွင်း ဖန်တီးထားတဲ့ scratchpad ဖိုင်ကို ရှင်းလင်းပါ:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# အကျဉ်းချုပ်

ဂုဏ်ယူပါတယ်! သင်အောင်မြင်စွာ အဆင့်မြင့်အကြောင်းအရာစီမံခန့်ခွဲမှုစွမ်းရည်များပါဝင်သော AI ကိုယ်စားလှယ်တစ်ဦးကို အကောင်အထည်ဖော်နိုင်ခဲ့ပါပြီ။

## သင်လေ့လာခဲ့တာများ:
- **Chat History Reduction**: Token အကန့်အသတ်များကို စီမံရန် စကားဝိုင်းများကို အလိုအလျောက် အကျဉ်းချုပ်ခြင်း
- **Agent Scratchpad**: အသုံးပြုသူ၏ အကြိုက်နှင့် ပြီးစီးထားသော အလုပ်များအတွက် အမြဲတမ်းမှတ်ဉာဏ်ကို အကောင်အထည်ဖော်ခြင်း
- **Token Management**: ရှည်လျားသော စကားဝိုင်းများတွင် Token အသုံးပြုမှုကို စောင့်ကြည့်ပြီး အကောင်းဆုံးဖြစ်အောင် ပြုလုပ်ခြင်း
- **Context Preservation**: စကားဝိုင်းအကျဉ်းချုပ်များအတွင်း အရေးကြီးသော အချက်အလက်များကို ထိန်းသိမ်းထားခြင်း

## အမှန်တကယ်အသုံးချနိုင်သော လုပ်ဆောင်ချက်များ:
- **Customer Service Bots**: အစည်းအဝေးများအတွင်း သုံးစွဲသူ၏ အကြိုက်များကို မှတ်မိထားခြင်း
- **Personal Assistants**: လက်ရှိစီမံကိန်းများနှင့် အသုံးပြုသူ၏ အလေ့အထများကို စောင့်ကြည့်ခြင်း
- **Educational Tutors**: ကျောင်းသား၏ တိုးတက်မှုနှင့် သင်ယူမှု အကြိုက်များကို ထိန်းသိမ်းထားခြင်း
- **Healthcare Assistants**: Token အကန့်အသတ်များကို လေးလေးနက်နက် ထိန်းသိမ်းရင်း လူနာ၏ သမိုင်းကြောင်းကို မှတ်မိထားခြင်း

## နောက်ထပ်အဆင့်များ:
- ပိုမိုတိုးတက်သော scratchpad schemas များကို အကောင်အထည်ဖော်ပါ
- Multi-user အခြေအနေများအတွက် ဒေတာဘေ့စ်သိုလှောင်မှုကို ထည့်သွင်းပါ
- နယ်ပယ်အထူးလိုအပ်ချက်များအတွက် အကျဉ်းချုပ်မူဝါဒများကို ဖန်တီးပါ
- Semantic memory ရှာဖွေရေးအတွက် vector databases နှင့် ပေါင်းစပ်ပါ
- ရက်များကြာပြီးနောက်လည်း အပြည့်အစုံသော အကြောင်းအရာဖြင့် စကားဝိုင်းများကို ပြန်လည်စတင်နိုင်သော ကိုယ်စားလှယ်များကို တည်ဆောက်ပါ



---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်ရန် လိုအပ်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူကောင်းမွန်သော ပရော်ဖက်ရှင်နယ်ဘာသာပြန်ဝန်ဆောင်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပါယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
